In [2]:
### Import packages for authentication

from google.cloud import bigquery
from google.oauth2 import service_account

### Import packages for converting query results into dataframe

import pandas as pd 

## Import packages to create absolute file path &  make code independent of operating system

from pathlib import Path
import os.path

### Import packages to visualize data

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
### Authentication 

base_path = Path("__file__").parent
full_path = (base_path / "../../data/raw/GoogleBigQuery_key.json").resolve()

credentials = service_account.Credentials.from_service_account_file(os.path.join(full_path))

In [ ]:
### Construct a BigQuery client object.

client = bigquery.Client(credentials=credentials, 
project = credentials.project_id)

In [ ]:
### Getting overview of Stackoverflow tables

stackoverflow = client.dataset('stackoverflow', project= 'bigquery-public-data')
print([x.table_id for x in client.list_tables(stackoverflow)])

In [ ]:
### Make an API request

query = """
SELECT
      pq.score, pq.id as question_id, pa.parent_id as question_id_check, pq.title as question_title, pq.body as question_text, pq.answer_count,
      pq.comment_count, pq.creation_date, pq.tags, pq.view_count, pa.body as answer_text
FROM `bigquery-public-data.stackoverflow.posts_questions` pq
INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` pa ON pq.id = pa.parent_id
WHERE pa.creation_date > "2019-05-30 00:00:00.000 UTC"
"""

dataframe = (
    client.query(query)
    .result()
    .to_dataframe()
)

In [4]:
len(dataframe)

2463032

In [ ]:
### Save dataframe to a csv file

base_path = Path("__file__").parent
full_path = (base_path / "../../data/raw/stackoverflow_raw.csv").resolve()

dataframe.to_csv(os.path.join(full_path))

In [4]:
### Temporary code to quickly open dataset

dataframe = pd.read_csv("/Users/HenriekeMax/Documents/Career_Development/GitHub/Predicting-Helpfulness-Of-Stackoverflow-Answers/data/raw/stackoverflow_raw.csv")

In [3]:
### Display query results

display(dataframe)

,Unnamed: 0,score,question_id,question_id_check,question_title,question_text,answer_count,comment_count,creation_date,tags,view_count,answer_text
0,0,0,56691024,56691024,Laravel query fails but works in Mysql,<p>I am new to Laravel and using the following...,1,2,2019-06-20 17:21:09.413000+00:00,mysql|laravel|date,52,<p>I think its because your config database's ...
1,1,0,56609735,56609735,How to find the distance between points from t...,<p>I was looking here for the answer but o did...,2,2,2019-06-15 11:02:43.897000+00:00,python|distance|haversine,55,<p>Here is an example using <code>iterrows</co...
2,2,1,11120176,11120176,Handle Recent apps click and home press in and...,<p>I am making an app for kids where once you'...,3,0,2012-06-20 13:01:50.997000+00:00,java|android|security|android-manifest,2221,<p>This is working like charm since API 14; te...
3,3,0,58028556,58028556,How to send an argument with an addEventListen...,<p>I'm quite new to coding and I ran into this...,3,2,2019-09-20 12:37:42.937000+00:00,javascript|arguments|addeventlistener,38,<p>You have a lot of options:</p>\n\n<p>1) Use...
4,4,-4,57951169,57951169,How to get the specified key value in JQuery/JS,<blockquote>\n <p>console.log(data)</p>\n</bl...,2,2,2019-09-16 06:05:34.393000+00:00,javascript|jquery,39,<p>Use <code>JSON.parse(data.LoginDetails).ip_...
...,...,...,...,...,...,...,...,...,...,...,...,...
2463027,2463027,0,16889968,16889968,adding shapelib to Qt's project,"<p>can you help me, i'm working under ubuntu O...",2,0,2013-06-03 04:20:31.830000+00:00,qt|shapefile,663,<p>If shapelib is installed in your system the...
2463028,2463028,0,61614244,61614244,not able to get array in the correct shape,<p>I am trying to create an elasticsearch quer...,1,0,2020-05-05 13:24:08.040000+00:00,php,38,<p>I think the extra index that you create for...
2463029,2463029,1,61658245,61658245,How do I to know on which GameObject I clicked...,<p>I want to make a Simple Memory Game using C...,4,0,2020-05-07 12:48:08.220000+00:00,c#|unity3d|2d|gameobject,44,<p>You could use RaycastHit to trap the GameOb...
2463030,2463030,0,57425396,57425396,How to determine the end of string in VFP 9?,"<p>In some programming languages, such as C fo...",2,0,2019-08-09 07:32:22.307000+00:00,string|visual-foxpro,151,<p>VFP strings are not ASCIIZ strings as in C....


In [5]:
### Check for null values

dataframe.isnull().values.any()
dataframe.isnull().sum()

Unnamed: 0           0
score                0
question_id          0
question_id_check    0
question_title       0
question_text        0
answer_count         0
comment_count        0
creation_date        0
tags                 1
view_count           0
answer_text          0
dtype: int64

In [6]:
### Dropping null value from tags column, given that it has only one missing value

dataframe = dataframe[dataframe['tags'].notnull()]
dataframe = dataframe.reset_index()[dataframe.columns]

In [ ]:
#### Histogram of distribution of the number of views of the question & answer

f, ax = plt.subplots(figsize=(40,40))
sns.countplot(x='view_count', data=dataframe)
plt.xlim(None, 200) 
plt.ylim(0, None) 
plt.show()

In [ ]:
### Summarization of variables for only numeric

dataframe.describe()

# mean score is 7.6 while 50% percentile / median score is 0.
# 25th percentile for view count is 39 views, median is 64 views;

In [7]:
dataframe['score'].value_counts() # .sort_index()

 0       1167343
 1        497483
 2        190425
-1        151273
 3         84687
          ...   
 1143          1
 1139          1
 1132          1
 1120          1
 689           1
Name: score, Length: 1612, dtype: int64

In [7]:
### Drop all datarows with answers that have gotten fewer than 39 views

dataframe_views = dataframe.drop(dataframe[dataframe.view_count < 39].index)
len(dataframe_views)

1849906

In [8]:
### Save dataframe with only > 39 to a csv file

base_path = Path("__file__").parent
full_path = (base_path / "../../data/raw/stackoverflow_raw_views.csv").resolve()

dataframe_views.to_csv(os.path.join(full_path))

In [10]:
### Assign category (bad, good, great) for score

score_bucketing_all = lambda x: 'bad' if x < 0 else 'good' if (x >= 1 and x <= 4) else 'great' if x >= 5 else None

dataframe_views['score_cat_all'] = dataframe_views['score'].apply(score_bucketing_all)

In [11]:
print("{} unique values in column".format('score_cat_all'))
print("{}".format(dataframe_views['score_cat_all'].unique()),"\n")

score_cat_all unique values in column
[None 'good' 'bad' 'great'] 



In [12]:
### Assign category (ood, great) for score; ignore negative and zero scores

score_bucketing_positive = lambda x: 'good' if (x >= 1 and x <= 4) else 'great' if x >= 5 else None

dataframe_views['score_cat_positive'] = dataframe_views['score'].apply(score_bucketing_positive)

In [13]:
print("{} unique values in column".format('score_cat_positive'))
print("{}".format(dataframe_views['score_cat_positive'].unique()),"\n")

score_cat_positive unique values in column
[None 'good' 'great'] 



In [14]:
### Look at the data to understand the type and missing values

print(dataframe_views.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1849906 entries, 0 to 2463030
Data columns (total 14 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   Unnamed: 0          int64 
 1   score               int64 
 2   question_id         int64 
 3   question_id_check   int64 
 4   question_title      object
 5   question_text       object
 6   answer_count        int64 
 7   comment_count       int64 
 8   creation_date       object
 9   tags                object
 10  view_count          int64 
 11  answer_text         object
 12  score_cat_all       object
 13  score_cat_positive  object
dtypes: int64(7), object(7)
memory usage: 211.7+ MB
None


In [15]:
display(dataframe_views)

,Unnamed: 0,score,question_id,question_id_check,question_title,question_text,answer_count,comment_count,creation_date,tags,view_count,answer_text,score_cat_all,score_cat_positive
0,0,0,56691024,56691024,Laravel query fails but works in Mysql,<p>I am new to Laravel and using the following...,1,2,2019-06-20 17:21:09.413000+00:00,mysql|laravel|date,52,<p>I think its because your config database's ...,None,None
1,1,0,56609735,56609735,How to find the distance between points from t...,<p>I was looking here for the answer but o did...,2,2,2019-06-15 11:02:43.897000+00:00,python|distance|haversine,55,<p>Here is an example using <code>iterrows</co...,None,None
2,2,1,11120176,11120176,Handle Recent apps click and home press in and...,<p>I am making an app for kids where once you'...,3,0,2012-06-20 13:01:50.997000+00:00,java|android|security|android-manifest,2221,<p>This is working like charm since API 14; te...,good,good
4,4,-4,57951169,57951169,How to get the specified key value in JQuery/JS,<blockquote>\n <p>console.log(data)</p>\n</bl...,2,2,2019-09-16 06:05:34.393000+00:00,javascript|jquery,39,<p>Use <code>JSON.parse(data.LoginDetails).ip_...,bad,None
5,5,1,61865206,61865206,springboot 2.3.0 while connecting to h2 database,<p>In Springboot 2.3.0.RELEASE\nI am getting t...,1,2,2020-05-18 08:12:27.733000+00:00,spring|database|spring-boot|h2,80,<p>You can fix this by setting the <code>sprin...,good,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463025,2463026,0,56929330,56929330,NiFi how to get the response of splitjson(each...,<p>I have the below sequence for calling rest ...,1,0,2019-07-08 06:21:33.223000+00:00,apache-nifi,207,<p>It sounds like you're trying to enrich the ...,None,None
2463026,2463027,0,16889968,16889968,adding shapelib to Qt's project,"<p>can you help me, i'm working under ubuntu O...",2,0,2013-06-03 04:20:31.830000+00:00,qt|shapefile,663,<p>If shapelib is installed in your system the...,None,None
2463028,2463029,1,61658245,61658245,How do I to know on which GameObject I clicked...,<p>I want to make a Simple Memory Game using C...,4,0,2020-05-07 12:48:08.220000+00:00,c#|unity3d|2d|gameobject,44,<p>You could use RaycastHit to trap the GameOb...,good,good
2463029,2463030,0,57425396,57425396,How to determine the end of string in VFP 9?,"<p>In some programming languages, such as C fo...",2,0,2019-08-09 07:32:22.307000+00:00,string|visual-foxpro,151,<p>VFP strings are not ASCIIZ strings as in C....,None,None


In [16]:
### Sample equal amounts of data rows for "bad", "good" and "great" answers from main dataset

dataframe_sampled_all = dataframe_views.groupby('score_cat_all', group_keys=False).apply(lambda x: x.sample(n=10000, random_state = 1)).reset_index(drop=True)

In [17]:
### Check if equal sampling of "bad", "good", and "great" answers was successful

dataframe_sampled_all.score_cat_all.value_counts()

good     10000
bad      10000
great    10000
Name: score_cat_all, dtype: int64

In [18]:
### Save dataframe to a csv file

base_path = Path("__file__").parent
full_path = (base_path / "../../data/raw/stackoverflow_raw_views_sampled_all.csv").resolve()

dataframe_sampled_all.to_csv(os.path.join(full_path))

In [25]:
### Sample only one answer randomly per stackoverflow question

df_all_final = dataframe_sampled_all.groupby('question_id', group_keys=False).apply(lambda x: x.sample(n = 1, random_state = 1)).reset_index(drop=True)

In [26]:
### Check if classes "bad", "good", and "great" are still roughly balanced

df_all_final.score_cat_all.value_counts()

good     9938
great    9705
bad      9686
Name: score_cat_all, dtype: int64

In [27]:
### Save sample of dataframe to a csv file

base_path = Path("__file__").parent
full_path = (base_path / "../../data/raw/stackoverflow_raw_finalsample_all.csv").resolve()
df_all_final.to_csv(os.path.join(full_path))

In [19]:
### Sample equal amounts of data rows for "good" and "great" answers from main dataset

dataframe_sampled_positive = dataframe_views.groupby('score_cat_positive', group_keys=False).apply(lambda x: x.sample(n=15000, random_state = 1)).reset_index(drop=True)

In [20]:
### Check if equal sampling of classes "good" and "great" was successful

dataframe_sampled_positive.score_cat_positive.value_counts()

good     15000
great    15000
Name: score_cat_positive, dtype: int64

In [21]:
### Save dataframe to a csv file

base_path = Path("__file__").parent
full_path = (base_path / "../../data/raw/stackoverflow_raw_views_sampled_positive.csv").resolve()

dataframe_sampled_positive.to_csv(os.path.join(full_path))

In [28]:
### Sample only one answer randomly per stackoverflow question

df_positive_final = dataframe_sampled_positive.groupby('question_id', group_keys=False).apply(lambda x: x.sample(n = 1, random_state = 1)).reset_index(drop=True)

In [30]:
### Check if classses "good" and "great" are still roughly balanced

df_positive_final.score_cat_positive.value_counts()

good     14856
great    14364
Name: score_cat_positive, dtype: int64

In [31]:
### Save dataframe to a csv file

base_path = Path("__file__").parent
full_path = (base_path / "../../data/raw/stackoverflow_raw_finalsample_positive.csv").resolve()

df_positive_final.to_csv(os.path.join(full_path))